In [1]:
#Instalação
!pip install pymupdf wget

     |████████████████████████████████| 18.6 MB 15.2 MB/s eta 0:00:01
  Using cached wget-3.2-py3-none-any.whl
You should consider upgrading via the '/Users/carlosalberto/Documents/projects/ai/TEESI_2025/venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import fitz
import os

import numpy as np
import pandas as pd

import re

In [ ]:
#Loader PDF to Text
class PdfToText():
  def __init__(self):
    pass

  def convert_from_file(self, filename):
    try:
      doc = self.load_pdf(filename)
    except:
      print( "Unable to open file." )
      return ""

    text = ""
    i = 0;
    for page in doc:
      text += page.get_text()+f"pagina {i}" + (50 * "-") + "\n\n\n"
      i = i + 1;
    
    doc.close()
    return text

  def load_pdf(self, filename):
    return fitz.open(filename)

class EnemExam(PdfToText):
  def __init__(self):
    pass

  def read_questions(self, filename):
    self.text = self.convert_from_file(filename)
    #print(self.text)
    self.find_topics()
    self.find_questions()

  def find_topics(self):
    self._topics = ['matemática e suas tecnologias',
                    'linguagens, códigos',
                    'ciências humanas',
                    'ciências da natureza',
                    ]

    topic_poss = [ self.text.lower().rfind(topic) for topic in self._topics ]

    self._topics_loc = [ (topic,topic_pos) for topic, topic_pos in zip(self._topics, topic_poss) if topic_pos>0 ]
    #ordena pela posição do tópico no texto.
    self._topics_loc.sort( key=lambda t: t[1] )

  def find_questions(self):
    matchs = []
    for match in re.finditer('\n(questão) [0-9]+', self.text.lower()):
      matchs.append( match.start() )

    start_pos = matchs[-1]
    match_end = re.search(r'\*[a-z0-9]+\*', self.text.lower()[start_pos:])
    matchs.append(start_pos + match_end.start())
   
    self.questions = []
    self.questions_topic = []
    self.questions_start = []
    for i in range( 1, len(matchs) ):
      self.questions.append( self.text[matchs[i-1]:matchs[i] ] )
      self.questions_topic.append( self.define_question_topic(matchs[i-1]) )
      self.questions_start.append( matchs[i-1] )

    
    print(self.questions[-1])

  def define_question_topic(self, start):
    question_topic = 0 #UNDEFINED

    for topic, topic_start in self._topics_loc:
      if start > topic_start:
        question_topic = (self._topics.index(topic)+1)

    return question_topic

In [83]:
nome_arquivo_provas = os.listdir('./temp/')
enem_exam = EnemExam()
enem_dfs = []

for nome_arquivo_prova in nome_arquivo_provas:
  enem_exam.read_questions('./temp/'+nome_arquivo_prova)
#  enem_dfs.append(
#      pd.DataFrame( data=dict( enunciado=enem_exam.questions, label=enem_exam.questions_topic )
#    )
#  )

[3677, 4165, 4800, 6960, 8067, 8499, 12006, 13074, 14694, 15965, 18763, 20554, 22409, 25456, 26790, 27900, 31325, 33106, 35106, 38254, 40592, 42008, 44824, 46479, 48067, 51370, 52613, 56178, 57742, 58858, 61237, 62702, 64224, 68239, 69543, 72150, 75357, 76757, 77227, 80741, 82351, 84230, 85892, 86428, 90043, 91250, 92642, 95645, 96891, 98174, 106951, 108314, 109140, 110018, 112597, 113734, 114752, 115765, 117821, 118781, 119559, 120641, 122976, 124564, 125424, 126576, 129174, 130514, 131307, 132744, 135112, 136147, 136970, 139848, 140794, 141719, 143274, 145944, 146725, 147901, 149200, 151675, 152126, 152927, 154549, 156131, 156907, 158072, 160382, 161816, 162927, 164968, 165796, 166678, 167577, 168359]

QUESTÃO 90	
Uma fábrica na qual os operários fossem, efetiva e 
integralmente, simples peças de máquinas executando 
cegamente as ordens da direção pararia em quinze minutos. 
O capitalismo só pode funcionar com a contribuição constante 
da atividade propriamente humana de seus subjuga

In [9]:
from sklearn.model_selection import train_test_split

questions_df = pd.concat( enem_dfs )


train_df, test_df = train_test_split(questions_df, test_size=0.2, random_state=42)

print(len(train_df))
print(len(test_df))

732
183


In [10]:
train_df.to_json( 'data/train/enem_questoes.json', orient='records' )
test_df.to_json( 'data/test/enem_questoes.json', orient='records' )

In [11]:
from datasets import load_dataset

# Caminho para o diretorio
json_dir_path = "./data"

# Carregar o dataset a partir do diretorio
dataset = load_dataset("json", data_dir= json_dir_path)

# Exibir algumas amostras do dataset
print(dataset)

/opt/anaconda3/envs/mestrado/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 732 examples [00:00, 17037.43 examples/s]
Generating test split: 183 examples [00:00, 28965.53 examples/s]

DatasetDict({
    train: Dataset({
        features: ['enunciado', 'label'],
        num_rows: 732
    })
    test: Dataset({
        features: ['enunciado', 'label'],
        num_rows: 183
    })
})


In [16]:
import pandas as pd

dataset.set_format(type="pandas")
df = dataset["train"][:300]

#df.head()

print(df.iloc[0]["enunciado"])


QUESTÃO 151	
Definem-se o dia e o ano de um planeta de um sistema solar 
como sendo, respectivamente, o tempo que o planeta leva para 
dar 1 volta completa em torno de seu próprio eixo de rotação e o 
tempo para dar 1 volta completa em torno de seu Sol.
Suponha que exista um planeta Z, em algum sistema 
solar, onde um dia corresponda a 73 dias terrestres e 
que 2 de seus anos correspondam a 1 ano terrestre. 
Considere que 1 ano terrestre tem 365 de seus dias.
No planeta Z, seu ano corresponderia a quantos de seus 
dias?
A	
A	 2,5
B	
B	 10,0
C	
C	 730,0
D	
D	 13 322,5
E	
E	 53 290,0
